In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib
import nbimporter
from functions import *
from numpy import *
import matplotlib.pyplot as plt
%matplotlib inline
import shutil
import os
import subprocess

In [ ]:
D7_dir='/home/mahdi/Desktop/data_selection_D7'
n,name=count(D7_dir)

out_dir='/home/mahdi/Desktop/valid'
n1,name1=count(out_dir)

In [ ]:
maximum_intensity=1705
model='200epoch_with_val.h5'
reference=0

for i in range(1,n):
    x=(len(name[i][0][:])-7)
    source_centerline_directory = '/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/centerline_volume0000.csv'
    centerlines_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/'
    main_data_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    center_fix_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_c.'+name[i][0][-6:]
    final_cplus_directory='/home/mahdi/Desktop/'
    mean_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/mean'
    

    cplus(source_centerline_directory,centerlines_directory,main_data_directory,
            center_fix_directory,final_cplus_directory,
            maximum_intensity,
          model,
          reference,
          mean_directory)

# 1- make structure

In [ ]:
if not os.path.exists('/home/mahdi/Desktop/centerline'):
     os.makedirs('/home/mahdi/Desktop/centerline')

for i in range(1,82):
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i))
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i)+'/3D_data')
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i)+'/center')
    
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i)+'/output/mid_ref')
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i)+'/output/zero_ref')
    os.makedirs('/home/mahdi/Desktop/centerline/'+str(i)+'/output/mean_ref')
    
    
   

# 2- copy files

In [ ]:
for i in range(n):
    x=(len(name[i][0][:])-7)
    input_mid='/home/mahdi/Desktop/data_selection_D7/'+name[i][0]
    dest_folder='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data'
    shutil.copy(input_mid, dest_folder)

# 3- make 3D data


In [ ]:
for i in range(n):
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    output_direction='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/vol'

    subprocess.Popen(['fslsplit',data_dir,output_direction,'-t']) ## just change the output names.

# 4- calculate centerline

In [ ]:
for k in range(1,n):
    data_dir ='/home/mahdi/Desktop/centerline/'+name1[k][0]+'/3D_data/'
    count,file_names=count_startwith(data_dir,'vol')
    for j in range(count):
        output_dir='/home/mahdi/Desktop/centerline/'+name1[k][0]+'/center/'
        input_dir=data_dir+file_names[j][0]
        subprocess.Popen(['sct_get_centerline','-i',input_dir,'-o',output_dir+'centerline_volume'+file_names[j][0][3:7],'-c','t2s']) ## just change the output names.


In [ ]:
name1[i][0]

# 5- shift image

### make reference

In [ ]:
#i=8
for i in range(1):
    Xs=[]
    Ys=[]
    source = pd.read_csv('/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/centerline_volume0000.csv', header=None)
    
    source.columns=['x','y','delete']
    source = source[['x','y']]

    for s in range(source.shape[0]):
        c=source.loc[s]
        xs=int(c['x'])
        ys=int(c['y'])
        Xs.append(xs)
        Ys.append(ys)

    n2,name2=count_endwith('/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center','.csv')

    dx=[]
    dy=[]
    for s in range(0,source.shape[0]):
         for j in range(n2):
                df = pd.read_csv('/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/'+name2[j][0], header=None)
                df.columns=['x','y','delete']
                df=df[['x','y']]
                c=df.loc[s]
                x=int(c['x'])
                y=int(c['y'])

                #xs=int((xs+x)/2)
                #xs=int((xs+x)/2)

                dx.append(Xs[s]-x)
                dy.append(Ys[s]-y)  
                
    input_direction='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    img  = nib.load(input_direction)
    img_data=img.get_fdata()
    img_mask_affine = img.affine
    header = img.header
    nb_img = header.get_data_shape()
    o=np.zeros((nb_img[0],nb_img[1],nb_img[2],nb_img[3]))


    DX=np.zeros(len(dy))


    start=0            
    for s in range(0,source.shape[0]):
        for v in range(n2):
                    a= shift_image(img_data[:,:,s,v],dy[v+start],DX[v+start])
                    o[:,:,s, v] = a            
        start=start + n2

    x=(len(name[i][0][:])-7)
    input_direction='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_c.'+name[i][0][-6:]
    img_reg = nib.Nifti1Image(o, affine=img_mask_affine, header=header)
    nib.save(img_reg,input_direction)
    
    
    maximum_intensity=1705
    model='200epoch_with_val.h5'
    output_direction_mean='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    reference='0'
    main(input_direction,reference,output_direction_mean,maximum_intensity,model)


In [ ]:
maximum_intensity=1705
model='200epoch_with_val.h5'
reference=0

for i in range(1,n):
    x=(len(name[i][0][:])-7)
    source_centerline_directory = '/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/centerline_volume0000.csv'
    centerlines_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/'
    main_data_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    center_fix_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_c.'+name[i][0][-6:]
    final_cplus_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    mean_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/mean'
    

    cplus(source_centerline_directory,centerlines_directory,main_data_directory,
            center_fix_directory,final_cplus_directory,
            maximum_intensity,
          model,
          reference,
          mean_directory)

In [ ]:
maximum_intensity=1705
model='200epoch_with_val.h5'
reference=-1

for i in range(59,n):
    x=(len(name[i][0][:])-7)
    source_centerline_directory = '/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/centerline_volume0000.csv'
    centerlines_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/'
    main_data_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    center_fix_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/'+name[i][0][:x]+'_c.'+name[i][0][-6:]
    final_cplus_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    mean_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/mean'
    

    cplus(source_centerline_directory,centerlines_directory,main_data_directory,
            center_fix_directory,final_cplus_directory,
            maximum_intensity,
          model,
          reference,
          mean_directory)

# make mean data

In [ ]:
for i in range(40,n):
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/mean.nii'
    
    
    subprocess.Popen(['sct_maths','-i',data_dir,'-mean','t' ,'-o',out ]) 

In [ ]:
maximum_intensity=1705
model='200epoch_with_val.h5'
reference=-2

for i in range(n):
    x=(len(name[i][0][:])-7)
    source_centerline_directory = '/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/centerline_volume0000.csv'
    centerlines_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/center/'
    main_data_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/3D_data/'+name[i][0]
    center_fix_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/'+name[i][0][:x]+'_c.'+name[i][0][-6:]
    final_cplus_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    mean_directory='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/mean.nii'
    

    cplus(source_centerline_directory,centerlines_directory,main_data_directory,
            center_fix_directory,final_cplus_directory,
            maximum_intensity,
          model,
          reference,
          mean_directory)

# TSNR for mid-ref

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

# TSNR for zero-ref

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

# TSNR for mean-ref

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/tsnr.nii'
    
    
    subprocess.Popen(['sct_fmri_compute_tsnr','-i',data_dir,'-o',out ]) 

# mask

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 

In [ ]:
for i in range(40,n):
    second3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/tsnr.nii'

    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/csf_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 
    
    input3_3='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_part.nii'
    output3_3='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/spine_mask.nii'
    subprocess.Popen(['sct_maths','-i',input3_3,'-mul',second3_3 ,'-o', output3_3]) 

# result

In [ ]:
csf_main1=[]
csf_sct1 =[]
csf_mine1=[]
csf_mine2=[]
csf_mine3=[]
spine_main1=[]
spine_mine1=[]
spine_mine2=[]
spine_mine3=[]
spine_sct1 =[]
start_time=time.time()

for i in range(n):
    output_sct_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/csf_mask.nii'
    output_main_csf='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/csf_mask.nii'
    output_mine_zeroref_csf='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/csf_mask.nii'
    output_mine_midref_csf='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/csf_mask.nii'
    output_mine_meanref_csf='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/csf_mask.nii'


    
    output_sct_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/sct_seg/spine_mask.nii'
    output_main_spine='/home/mahdi/Desktop/valid/'+name1[i][0]+'/main_seg/spine_mask.nii'
    output_mine_zeroref_spine='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/spine_mask.nii'
    output_mine_midref_spine='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/spine_mask.nii'
    output_mine_meanref_spine='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/spine_mask.nii'

    
    
    
    csf_main=mean_all(output_main_csf)
    csf_main1.append(csf_main)
    csf_sct=mean_all(output_sct_csf)
    csf_sct1.append(csf_sct)
    
    csf_mine=mean_all(output_mine_zeroref_csf)
    csf_mine1.append(csf_mine)
    csf_mine_mid=mean_all(output_mine_midref_csf)
    csf_mine2.append(csf_mine_mid)
    csf_mine_mean=mean_all(output_mine_meanref_csf)
    csf_mine3.append(csf_mine_mean)
    
    
    spine_main=mean_all(output_main_spine)
    spine_main1.append(spine_main)
    spine_sct=mean_all(output_sct_spine)
    spine_sct1.append(spine_sct)
    
    
    spine_mine=mean_all(output_mine_zeroref_spine)
    spine_mine1.append(spine_mine)
    spine_mine_mid=mean_all(output_mine_midref_spine)
    spine_mine2.append(spine_mine_mid)
    spine_mine_mean=mean_all(output_mine_meanref_spine)
    spine_mine3.append(spine_mine_mean)

    
    

csf_main=np.mean(csf_main1)
csf_sct=np.mean(csf_sct1)
csf_mine_zero=np.mean(csf_mine1)
csf_mine_mid=np.mean(csf_mine2)
csf_mine_mean=np.mean(csf_mine3)




spine_main=np.mean(spine_main1)
spine_sct=np.mean(spine_sct1)
spine_mine_zero=np.mean(spine_mine1)
spine_mine_mid=np.mean(spine_mine2)
spine_mine_mean=np.mean(spine_mine3)

print("--- %s second ---" % (time.time() - start_time))

    
pd.DataFrame([csf_main,csf_sct,csf_mine_zero,csf_mine_mid,csf_mine_mean,spine_main,spine_sct,spine_mine_zero,spine_mine_mid,spine_mine_mean],
         index=['main_csf_tsnr', 'sct_csf_tsnr', 'my_csf_tsnr_first','my_csf_tsnr_mid','my_csf_tsnr_mean',
                'main_spine_tsnr','sct_spine_tsnr','my_spine_tsnr_first','my_spine_tsnr_mid','my_spine_tsnr_mean']
        )

# sort in row

In [ ]:
#final=np.array([csf_main1 , csf_sct1,csf_mine1,csf_mine2,csf_mine3,
                spine_main1,spine_sct1,spine_mine1,spine_mine2,spine_mine3])
#out_dir='/home/mahdi/Desktop/result/tsnr_centerfix_result.csv'
#pd.DataFrame(final,index=['csf_nomoco','csf_sct','csf_myalgo_zero_ref','csf_myalgo_mid_ref','csf_myalgo_mean_ref','spine_nomoco','spine_sct','spine_myalgo_zero_ref','spine_myalgo_mid_ref','spine_myalgo_mean_ref']).to_csv(out_dir)

# sort in column

In [ ]:
final=pd.DataFrame([csf_main1 , csf_sct1,csf_mine1,csf_mine2,csf_mine3,
                spine_main1,spine_sct1,spine_mine1,spine_mine2,spine_mine3])

final_transpose=final.T
final_transpose.columns=['csf_nomoco','csf_sct','csf_myalgo_zero_ref','csf_myalgo_mid_ref','csf_myalgo_mean_ref','spine_nomoco','spine_sct','spine_myalgo_zero_ref','spine_myalgo_mid_ref','spine_myalgo_mean_ref']
outt_dir='/home/mahdi/Desktop/result/tsnr_centerfix_column_result.csv'
pd.DataFrame(final_transpose).to_csv(outt_dir)
final_transpose

# Dvars

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/zero_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mid_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

In [ ]:
for i in range(n):
    
    x=(len(name[i][0][:])-7)
    data_dir='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref/'+name[i][0][:x]+'_cplus.'+name[i][0][-6:]
    out='/home/mahdi/Desktop/centerline/'+name1[i][0]+'/output/mean_ref'
    subprocess.Popen(['fsl_motion_outliers','-i',data_dir,'-s',out+'/Dvars','-o',out+'/e' ]) 

# calculate result

In [4]:
main_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'main_seg'+'/Dvars'
    main_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_main_Dvars=np.mean(main_Dvars) 
pd.DataFrame(main_Dvars) 

,0
0,0.033274
1,0.027648
2,0.022165
3,0.044346
4,0.044644
...,...
76,0.052032
77,0.020753
78,0.028959
79,0.031971


In [5]:
sct_Dvars=[]
for j in range(n):
    data_dir=out_dir+'/'+name1[j][0]+'/'+'sct_seg'+'/Dvars'
    sct_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_sct_Dvars=np.mean(sct_Dvars)   
pd.DataFrame(sct_Dvars)  

,0
0,0.035907
1,0.025350
2,0.019106
3,0.042239
4,0.041873
...,...
76,0.049396
77,0.021202
78,0.028252
79,0.025799


In [6]:
my_zero_Dvars=[]
for j in range(n):
    data_dir='/home/mahdi/Desktop/centerline/'+name1[j][0]+'/output/zero_ref/Dvars'
    my_zero_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_zero_Dvars=np.mean(my_zero_Dvars)
pd.DataFrame(my_zero_Dvars)  
#pd.DataFrame(my_zero_Dvars).to_csv('/home/mahdi/Desktop/result/my_zero_Dvars.csv')

,0
0,0.022779
1,0.021928
2,0.019264
3,0.047516
4,0.033296
...,...
76,0.035477
77,0.016770
78,0.025652
79,0.050358


In [7]:
my_mid_Dvars=[]
for j in range(n):
    data_dir='/home/mahdi/Desktop/centerline/'+name1[j][0]+'/output/mid_ref/Dvars'
    my_mid_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_mid_Dvars=np.mean(my_mid_Dvars)
pd.DataFrame(my_mid_Dvars)  
#pd.DataFrame(my_zero_Dvars).to_csv('/home/mahdi/Desktop/result/my_zero_Dvars.csv')

,0
0,0.022225
1,0.021830
2,0.019980
3,0.047995
4,0.034826
...,...
76,0.038065
77,0.015886
78,0.023788
79,0.045621


In [17]:
my_mean_Dvars=[]
for j in range(n):
    data_dir='/home/mahdi/Desktop/centerline/'+name1[j][0]+'/output/mean_ref/Dvars'
    my_mean_Dvars.append(np.mean(pd.read_csv(data_dir)))
mean_my_mean_Dvars=np.mean(my_mean_Dvars)
#pd.DataFrame(my_mean_Dvars)  
#pd.DataFrame(my_zero_Dvars).to_csv('/home/mahdi/Desktop/result/my_zero_Dvars.csv')

In [10]:
pd.DataFrame([mean_main_Dvars,mean_sct_Dvars,mean_my_zero_Dvars,mean_my_mid_Dvars,mean_my_mean_Dvars],
            index=['mean_main_Dvars', 'mean_sct_Dvars', 'mean_my_zero_Dvars','mean_my_mid_Dvars','mean_my_mean_Dvars',
                ])

,0
mean_main_Dvars,0.034251
mean_sct_Dvars,0.031623
mean_my_zero_Dvars,0.028224
mean_my_mid_Dvars,0.029199
mean_my_mean_Dvars,0.028631


# row order

In [ ]:
#final_Dvars=np.array([main_Dvars ,sct_Dvars ,my_zero_Dvars ,my_mid_Dvars ,my_mean_Dvars])
#final_Dvars=final_Dvars[:,:,0]
#pd.DataFrame(final_Dvars,index=['main_Dvars','sct_Dvars','myalgo_zero_Dvars','myalgo_mid_Dvars','myalgo_mean_Dvars']).to_csv('/home/mahdi/Desktop/result/Dvars_centerfix_result.csv')


# column order

In [23]:
final_Dvars=np.array([main_Dvars ,sct_Dvars ,my_zero_Dvars ,my_mid_Dvars ,my_mean_Dvars])
final_Dvars=final_Dvars[:,:,0]
final_Dvars=pd.DataFrame(final_Dvars)

final_Dvars_transpose=final_Dvars.T
final_Dvars_transpose.columns=['main_Dvars','sct_Dvars','myalgo_zero_Dvars','myalgo_mid_Dvars','myalgo_mean_Dvars']
pd.DataFrame(final_Dvars_transpose).to_csv('/home/mahdi/Desktop/result/Dvars_centerfix_column_result.csv')



In [1]:
v=101
images = [img[v, :, :, 0] for img in val_input + val_pred] 
titles = ['moving', 'fixed', 'moved', 'flow']
ne.plot.slices(images, titles=titles, cmaps=['gray'], do_colorbars=True);


NameError: name 'val_input' is not defined

In [ ]:
ne.plot.flow([val_pred[1][v,...].squeeze()], width=5)